In [1]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client['polytest']
markets_col = db['markets']

df = pd.DataFrame(list(markets_col.find()))
# df.head()
# print(df.head())



In [2]:
# count all rows id's
df['_id'].count()
print("Rows:", df['_id'].count())

Rows: 8377


In [3]:
count = df['events'].apply(lambda ev: any('endDate' in e for e in ev) if isinstance(ev, list) else False).sum()
print("Count end dates:", count)

Count end dates: 8325


In [4]:
from datetime import datetime

# Target date
target_date = "2025-12-17"

# Flatten and filter events
end_dates = []
for events in df['events']:
    if isinstance(events, list):
        for e in events:
            if 'endDate' in e and e['endDate'] is not None:
                # Compare date portion only
                if e['endDate'].startswith(target_date):
                    end_dates.append(e['endDate'])
# print(end_dates)
# count end_dates
count = len(end_dates)
print("Total Dates:", target_date, count)

Total Dates: 2025-12-17 169


In [5]:
# display "title" of the target_date on limit 5 rows
# Filter rows where any event has endDate starting with target_date
mask = df['events'].apply(
    lambda ev: any(e.get('endDate', '').startswith(target_date) for e in ev) 
    if isinstance(ev, list) else False
)

# Extract titles from matching events
titles = [
    e.get('title')
    for events in df['events'] if isinstance(events, list)
    for e in events if e.get('endDate', '').startswith(target_date)
]
# for events in df[mask]['events']:
#     if isinstance(events, list):
#         for e in events:
#             if e.get('endDate', '').startswith(target_date):
#                 titles.append(e.get('title'))

# Show first 5 titles
print(titles[:5], target_date)

['Florida A&M Rattlers vs. Florida Gators (W)', 'Spurs vs. Knicks', 'Bitcoin Up or Down - December 17, 2AM ET', 'Solana price on December 17?', 'November Inflation Eurozone - Annual'] 2025-12-17


In [6]:
# dupicate id's check
# Find duplicate IDs
duplicate_ids = df[df.duplicated(subset='id', keep=False)]['id']

# Show unique duplicate IDs
print(duplicate_ids.unique())

# Count of duplicate IDs
print("Total duplicate IDs:", len(duplicate_ids.unique()))



[]
Total duplicate IDs: 0


In [7]:
# Flatten id and title pairs
id_title_pairs = []
for _, row in df.iterrows():
    events = row['events']
    if isinstance(events, list):
        for e in events:
            if 'title' in e:
                id_title_pairs.append({'id': row['id'], 'title': e['title']})

# Convert to DataFrame
id_title_df = pd.DataFrame(id_title_pairs)

# Find duplicates by title
duplicate_df = id_title_df[id_title_df.duplicated(subset='title', keep=False)]

# Show unique duplicates
print(duplicate_df)

# Save to CSV
duplicate_df.to_csv("duplicate_titles_with_ids.csv", index=False)
print("Saved duplicate titles with IDs to duplicate_titles_with_ids.csv")


          id                                              title
0     679162                 2026 Busan Mayoral Election Winner
1     678779  How many different countries will the US strik...
3     540208                           Super Bowl Champion 2026
4     679257          Illinois Democratic Senate Primary Winner
6     679036                  Next French Presidential Election
...      ...                                                ...
8368  928845      West Ham United FC vs. Queens Park Rangers FC
8369  898413                               F1 Drivers' Champion
8371  931892                  UC Sampdoria vs. AC Reggiana 1919
8374  937234       Dota 2: Xtreme Gaming vs Natus Vincere (BO3)
8375  907122       Tennessee Governor Democratic Primary Winner

[6344 rows x 2 columns]
Saved duplicate titles with IDs to duplicate_titles_with_ids.csv


In [8]:
# Flatten id, title, closed
rows = []

for _, row in df.iterrows():
    events = row.get('events')
    if isinstance(events, list):
        for e in events:
            if 'title' in e:
                rows.append({
                    'id': row['id'],
                    'title': e['title'],
                    'closed': e.get('closed', row.get('closed'))
                })

# Convert to DataFrame
id_title_df = pd.DataFrame(rows)

# Filter specific title
bitcoin_df = id_title_df[
    id_title_df['title'].str.contains('Bitcoin Up or Down', case=False, na=False)
]

print(bitcoin_df)
print("Count:", len(bitcoin_df))

# Save
bitcoin_df.to_csv("bitcoin_titles_with_ids.csv", index=False)
print("Saved bitcoin_titles_with_ids.csv")


          id                                              title  closed
35    936392           Bitcoin Up or Down - December 16, 4PM ET   False
207   933683  Bitcoin Up or Down - December 15, 12:30PM-12:4...   False
234   938432  Bitcoin Up or Down - December 15, 8:00PM-8:15P...   False
264   939426           Bitcoin Up or Down - December 17, 2AM ET   False
352   935803  Bitcoin Up or Down - December 15, 3:15PM-3:30P...   False
...      ...                                                ...     ...
8170  937677  Bitcoin Up or Down - December 15, 6:15PM-6:30P...   False
8180  938150  Bitcoin Up or Down - December 15, 7:00PM-7:15P...   False
8233  939700  Bitcoin Up or Down - December 16, 4:15AM-4:30A...   False
8289  939859  Bitcoin Up or Down - December 16, 4:30AM-4:45A...   False
8317  933692  Bitcoin Up or Down - December 15, 12:45PM-1:00...   False

[180 rows x 3 columns]
Count: 180
Saved bitcoin_titles_with_ids.csv
